In [1]:
import pandas as pd
import re
from tqdm import tqdm
import copy
from sklearn.model_selection import train_test_split

# Code to remove sentence and clean text

In [2]:
import re

# Importing contractions
with open("contractions.txt", "r") as inp_cont:
    contractions_list = inp_cont.read()

# Process contractions list and handle potential errors
contractions_list = [
    re.sub('["]', '', x).split(":") for x in re.sub(r"\s+", " ", re.sub(r"(.*{)|(}.*)", '', contractions_list)).split(',')
]

# Safely create contractions_dict and print warnings for improperly formatted items
contractions_dict = {}
for item in contractions_list:
    if len(item) == 2:
        k, v = item
        contractions_dict[k.lower().strip()] = re.sub('/.*', '', v).lower().strip()
    else:
        print(f"Skipping improperly formatted item: {item}")

def remove_sc(_line, lang="en"):
    # Removes special characters based on the language
    if lang == "hi":
        _line = re.sub(r'[+\-*/#@%>=;~{}×–`’"()_]', "", _line)
        _line = re.sub(r"(?:(\[)|(\])|(‘‘)|(’’))", '', _line)
    elif lang == "en":
        _line = re.sub(r'[+\-*/#@%>=;~{}×–`’"()_|:]', "", _line)
        _line = re.sub(r"(?:(\[)|(\])|(‘‘)|(’’))", '', _line)
    return _line

def clean_text(_text, lang="en"):
    # Clean the text using special character removal and contractions expansion
    _text = remove_sc(_line=_text, lang=lang)
    if lang == "en":
        for cn in contractions_dict:
            # Using word boundaries (\b) to avoid partial matches
            _text = re.sub(r'\b' + re.escape(cn) + r'\b', contractions_dict[cn], _text)
    return _text


Skipping improperly formatted item: ["ain't aren't can't could've couldn't couldn't've didn't doesn't don't hadn't hadn't've hasn't haven't he'd he'd've he'll he's how'd how'll how's I'd I'd've I'll I'm I've isn't it'd it'd've it'll it's let's ma'am mightn't mightn't've might've mustn't must've needn't not've o'clock oughtn't 'ow's'at shan't she'd she'd've she'll she's should've shouldn't shouldn't've somebody'd somebody'd've somebody'll somebody's someone'd someone'd've someone'll someone's something'd something'd've something'll something's that'll that's there'd there'd've there're there's they'd they'd've they'll they're they've 'twas wasn't we'd we'd've we'll we're we've weren't what'll what're what's what've when's where'd where's where've who'd who'd've who'll who're who's who've why'll why're why's won't would've wouldn't wouldn't've y'all y'all'll y'all'd've you'd you'd've you'll you're you've "]


In [3]:
df = pd.read_csv("hindi_english_parallel.csv")

hindi_text = df['hindi']
english_text = df['english']

In [4]:
print(hindi_text.head)
print(english_text.head)

<bound method NDFrame.head of 0            अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1                            एक्सेर्साइसर पहुंचनीयता अन्वेषक
2                      निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3                       ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4          उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
                                 ...                        
1561836    स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837    मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...
1561838    उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...
1561839    खाद्य और जल सुरक्षा; पर्यावरण की दृष्टि से वहन...
1561840                                                  NaN
Name: hindi, Length: 1561841, dtype: object>
<bound method NDFrame.head of 0             Give your application an accessibility workout
1                          Accerciser Accessibility Explorer
2             The default plugin layout for the bottom panel
3                The default plugin layou

In [5]:
ids_to_remove = {}
for _id, _t in tqdm(enumerate(hindi_text)):
    # Check if _t is a string, if not, skip or convert to string
    if isinstance(_t, str):
        if len(re.findall(r'[a-zA-Z]', _t)) > 0:
            ids_to_remove[_id] = _t
    else:
        pass

1561841it [00:07, 220077.46it/s]


In [ ]:
from tqdm import tqdm

# Assuming `ids_to_keep` and `ids_to_remove` are defined as before
ids_to_keep = [i for i in range(len(hindi_text)) if i not in ids_to_remove.keys()]

filtered_eng_text = []
filtered_hindi_text = []

# Iterate over the indices to keep
for _id in tqdm(ids_to_keep):
    eng_text_value = english_text[_id]
    
    # Check if the value is a string before calling .lower()
    if isinstance(eng_text_value, str):
        filtered_eng_text.append(eng_text_value.lower())
    else:
        # Handle non-string values (e.g., logging the issue)
        print(f"Non-string value encountered at index {_id}: {eng_text_value}")
        # Choose how to handle it; here we append an empty string
        filtered_eng_text.append("")  # or you can continue to skip this value
    
    # Append the corresponding Hindi text (assuming it's valid)
    filtered_hindi_text.append(hindi_text[_id])

# Optionally, print the results for verification
print("Filtered English Text:")
print(filtered_eng_text)
print("Filtered Hindi Text:")
print(filtered_hindi_text)

# Additional debugging: Print types of all elements in english_text
print("\nTypes of elements in english_text:")
for index, value in enumerate(english_text):
    print(f"Index {index}: Type {type(value)}, Value: {value}")


  1%|▏         | 20598/1434242 [00:00<00:14, 98063.63it/s]

Non-string value encountered at index 7838: nan
Non-string value encountered at index 8416: nan
Non-string value encountered at index 9082: nan
Non-string value encountered at index 9785: nan
Non-string value encountered at index 10492: nan
Non-string value encountered at index 12371: nan
Non-string value encountered at index 14466: nan
Non-string value encountered at index 16441: nan
Non-string value encountered at index 18295: nan
Non-string value encountered at index 20333: nan
Non-string value encountered at index 22281: nan
Non-string value encountered at index 24250: nan
Non-string value encountered at index 26205: nan


  3%|▎         | 48689/1434242 [00:00<00:18, 75526.76it/s]

Non-string value encountered at index 51968: nan
Non-string value encountered at index 53239: nan
Non-string value encountered at index 53533: nan
Non-string value encountered at index 54285: nan
Non-string value encountered at index 55267: nan
Non-string value encountered at index 56205: nan
Non-string value encountered at index 57756: nan
Non-string value encountered at index 58030: nan
Non-string value encountered at index 58321: nan
Non-string value encountered at index 58618: nan
Non-string value encountered at index 58927: nan
Non-string value encountered at index 59258: nan
Non-string value encountered at index 59590: nan
Non-string value encountered at index 59895: nan
Non-string value encountered at index 60206: nan
Non-string value encountered at index 60529: nan
Non-string value encountered at index 60848: nan
Non-string value encountered at index 61175: nan
Non-string value encountered at index 70038: nan
Non-string value encountered at index 70404: nan
Non-string value enc

  4%|▍         | 63595/1434242 [00:00<00:20, 66146.13it/s]

...

 71%|███████▏  | 1024028/1434242 [00:13<00:04, 91996.30it/s]

Non-string value encountered at index 1130790: nan


 73%|███████▎  | 1050000/1434242 [00:14<00:04, 77940.52it/s]

Non-string value encountered at index 1158016: nan


100%|██████████| 1434242/1434242 [00:19<00:00, 73769.86it/s]


In [15]:
#Treating english sentences
clean_eng_text = []
for sent in tqdm(filtered_eng_text):
    clean_eng_text.append(clean_text(_text=copy.deepcopy(sent), lang="en"))
    


100%|██████████| 904975/904975 [02:42<00:00, 5571.89it/s]


In [19]:
#Treating hindi sentences
clean_hindi_text = []
for sent in tqdm(filtered_hindi_text):
    clean_hindi_text.append(clean_text(_text=copy.deepcopy(sent), lang="hi"))

100%|██████████| 904975/904975 [00:08<00:00, 109276.45it/s]


In [20]:
clean_data = pd.DataFrame({'eng_text':clean_eng_text, 'hindi_text':clean_hindi_text})

In [21]:
clean_data.head

<bound method NDFrame.head of                                                  eng_text  \
0          give your application an accessibility workout   
1                       accerciser accessibility explorer   
2          the default plugin layout for the bottom panel   
3             the default plugin layout for the top panel   
4          a list of plugins that are disabled by default   
...                                                   ...   
904970  कते हैं। opening of a finnode centre in delhi ...   
904971  प्राप्त होगी। by 2020, india is set to become ...   
904972  है। i note that this is a landmark meeting  no...   
904973  है। in the presentations that they made before...   
904974  ्त है। issues such as food and water security ...   

                                               hindi_text  
0         अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें  
1                         एक्सेर्साइसर पहुंचनीयता अन्वेषक  
2                    निचले पटल के लिए डिफोल्ट प्लगइन खाका

## Filtering data based on sentence length

In [22]:
clean_data["eng_len"] = clean_data.eng_text.str.count("")
clean_data['hindi_len'] = clean_data.hindi_text.str.count("")
small_len_data = clean_data.query('eng_len < 50 & hindi_len < 50')

In [26]:
#Train_Val split
#Full set

train_set, val_set = train_test_split(small_len_data.loc[:, ['eng_text','hindi_text']], test_size=0.1)
train_set.to_csv("train.csv", index=False)
val_set.to_csv("val.csv", index=False)

In [27]:
small_data = small_len_data.loc[:, ["eng_text", "hindi_text"]].sample(n=150000)
train_set_sm, val_set_sm = train_test_split(small_data, test_size=0.3)
train_set_sm.to_csv("train_sm.csv", index=False)
val_set_sm.to_csv("val_sm.csv", index=False)